In [ ]:
%cd Dixon

In [1]:
import numpy as np
from numpy import inf
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.colors import Colormap
import scipy
from scipy import io
import glob
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
import aplpy
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import pyvo as vo
from urllib import urlretrieve
import copy
import re

def downloadTGSS(fitsRA, fitsDEC, framesize=.5, frame='icrs', NVSS=False, filename=None):

    """
    This is used to download a FITS image file of an extended object from TGSS or NVSS. It
    takes a .sav file and an object ID, and outputs the name of the FITS file that is downloaded.
    
    Args:
        data: A .sav file that has been run through clst.collector
        EOid (int): the 5-digit ID describing the extended object (must be from data)
        NVSS (boolean): 
            if True, downloads a FITS file of the object from NVSS
            if False, uses TGSS instead of NVSS (default is False)
            
    Returns: 
        filename: name of the FITS file which is downloaded, based on EOid, RA, and DEC.
    """
    
    from astropy.coordinates import SkyCoord
    import pyvo as vo
    from urllib import urlretrieve
    
    myLocation = SkyCoord(fitsRA*u.deg, fitsDEC*u.deg, frame = frame)

    if NVSS == False:
        query = vo.sia.SIAQuery('http://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=TGSS&',
                                pos=(myLocation.ra.deg, myLocation.dec.deg),
                                size = framesize, format='image/fits')
        if filename is None:
            filename = 'RA{}_DEC{}_TGSS.fits'.format(myLocation.ra.deg, myLocation.dec.deg)
        else:
            filename = filename
    else:
        query = vo.sia.SIAQuery('http://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=NVSS&',
                                pos=(myLocation.ra.deg, myLocation.dec.deg),
                                size = framesize, format='image/fits')
        if filename is None:
            filename = 'RA{}_DEC{}_NVSS.fits'.format(myLocation.ra.deg, myLocation.dec.deg)
        else:
            filename = filename
    print query
    results = query.execute()
    #now we extract the url of the fits file we want
    url = results[0].getdataurl()

    #and download it somewhwere. I’ve hardcoded the name, but you’ll want to have this name change
    urlretrieve(url, filename)

    fitsimage = '{}'.format(filename)
    
    fig = plt.figure(figsize=(8,8))
    fig.suptitle('')


    ax = aplpy.FITSFigure(fitsimage,figure=fig,)
    ax.show_colorscale(cmap='gray',vmin=0)   
    ax.add_colorbar()

    ax.show_scalebar(1)
    ax.scalebar.set_length(2 * u.arcminute)
    ax.scalebar.set_corner(corner='bottom right')
    ax.scalebar.set_label("2 arcminutes")
    ax.scalebar.set_color('blue')
    ax.scalebar.set_frame(True)

    ax.set_title('{}'.format(filename))
    ax.set_xaxis_coord_type('scalar')
    ax.set_yaxis_coord_type('scalar')
    ax.tick_labels.set_xformat('%11.3f')
    ax.tick_labels.set_yformat('%11.3f')


    ax = fig.gca()

    ax.patch.set_alpha(0.0)
    plt.savefig('{}.png'.format(filename))
    plt.show()

In [ ]:
print SkyCoord('00 42 32.3 +40 34 19', unit=(u.hourangle, u.deg))

In [ ]:
downloadTGSS(10.63458333,  40.57194444, framesize=.015, filename='AND IV')

In [ ]:
%cd Dixon
Chris = np.genfromtxt('combined-radii-visually-inspected-5-23-17.txt', \
                      names=True,delimiter=',',dtype=None,unpack=True,skip_header=0)

In [ ]:
for i in range(len(Chris['objid'])):
    downloadTGSS(Chris['ra'][i],  Chris['dec'][i], framesize=.05, filename='{}_framedeg{}'.format(Chris['objid'][i],.05))

In [ ]:
print np.where(Chris['objid']==1237667444048658450)

In [ ]:
print Chris['ra'][0]

In [ ]:
downloadTGSS(Chris['ra'][0],  Chris['dec'][0], framesize=.02, filename='smaller')

In [ ]:
services = vo.regsearch(servicetype='sia', waveband='x-ray')
for service in services:
    print(service.res_title)

In [ ]:
for service in services:
    print service.res_title, service.access_url

In [2]:
cats = vo.regsearch(servicetype='sia', waveband='radio')
for cat in cats:
    print cat.res_title, cat.access_url

CSIRO ASKAP Science Data Archive Image Access Service https://casda.csiro.au/casda_vo_tools/sia2/query?
Cosmic Evolution Survey with HST https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=COSMOS&hdr_location=%5CCOSMOSDataPath%5C&collection_desc=Cosmic+Evolution+Survey+with+HST+%28COSMOS%29&SIAP_ACTIVE=1&
VLA-A Array AL218 Texas Survey Source Snapshots (AL218) http://archive.stsci.edu/siap/search.php?id=al218&
VLA Faint Images of the Radio Sky at Twenty Centimeters (FIRST) http://archive.stsci.edu/siap/search.php?id=vla-first&
Bonn 1420 MHz Survey https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=1420mhz&
HI All-Sky Continuum Survey https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=408mhz&
LABOCA Extended Chandra Deep Field South Submillimetre Survey https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=cdfs-less&
CO Galactic Plane Survey https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=co&
Effelsberg-Bonn HI Survey https://skyview.gsfc.nasa.gov/cgi-bin/vo/si

In [3]:
print cats[1].access_url

https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=COSMOS&hdr_location=%5CCOSMOSDataPath%5C&collection_desc=Cosmic+Evolution+Survey+with+HST+%28COSMOS%29&SIAP_ACTIVE=1&


In [ ]:
def showVLA(fitsRA, fitsDEC, x, framesize=.5, frame='icrs', NVSS=False, filename=None):

    """
    This is used to download a FITS image file of an extended object from TGSS or NVSS. It
    takes a .sav file and an object ID, and outputs the name of the FITS file that is downloaded.
    
    Args:
        data: A .sav file that has been run through clst.collector
        EOid (int): the 5-digit ID describing the extended object (must be from data)
        NVSS (boolean): 
            if True, downloads a FITS file of the object from NVSS
            if False, uses TGSS instead of NVSS (default is False)
            
    Returns: 
        filename: name of the FITS file which is downloaded, based on EOid, RA, and DEC.
    """
    
    from astropy.coordinates import SkyCoord
    import pyvo as vo
    from urllib import urlretrieve
    
    myLocation = SkyCoord(fitsRA*u.deg, fitsDEC*u.deg, frame = frame)

    query = vo.sia.SIAQuery(cats[x].access_url,
                            pos=(myLocation.ra.deg, myLocation.dec.deg),
                            size = framesize, format='image/fits')
    if filename is None:
        filename = 'RA{}_DEC{}_VLA.fits'.format(myLocation.ra.deg, myLocation.dec.deg)
    else:
        filename = filename

    print query
    results = query.execute()
    #now we extract the url of the fits file we want
    url = results[0].getdataurl()

    #and download it somewhwere. I’ve hardcoded the name, but you’ll want to have this name change
    urlretrieve(url, filename)

    fitsimage = '{}'.format(filename)
    
    fig = plt.figure(figsize=(8,8))
    fig.suptitle('')


    ax = aplpy.FITSFigure(fitsimage,figure=fig,)
    ax.show_colorscale(cmap='gray',vmin=0)   
    ax.add_colorbar()

    ax.show_scalebar(1)
    ax.scalebar.set_length(2 * u.arcminute)
    ax.scalebar.set_corner(corner='bottom right')
    ax.scalebar.set_label("2 arcminutes")
    ax.scalebar.set_color('blue')
    ax.scalebar.set_frame(True)

    ax.set_title('{}'.format(filename))
    ax.set_xaxis_coord_type('scalar')
    ax.set_yaxis_coord_type('scalar')
    ax.tick_labels.set_xformat('%11.3f')
    ax.tick_labels.set_yformat('%11.3f')


    ax = fig.gca()

    ax.patch.set_alpha(0.0)
    plt.savefig('{}.png'.format(filename))
    plt.show()

In [ ]:
print SkyCoord('00 47 33.134 -25 17 19.68', unit=(u.hourangle, u.deg))

In [ ]:
showVLA(Chris['ra'][2],  Chris['dec'][2], 16, framesize=5)